# Research Service Workflow

# Overview
Build a research assistant that can search the web, analyze content, and provide comprehensive reports using Toolhouse.ai's pre-built tools.

In [2]:
# Install required packages
!pip install toolhouse openai python-dotenv pandas

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.8/43.8 kB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 264.0/264.0 kB 8.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.5/127.5 kB 4.4 MB/s eta 0:00:00


In [3]:
# Import dependencies
import os
import json
from dotenv import load_dotenv
from openai import OpenAI
from toolhouse import Toolhouse, Provider
import pandas as pd

# Load environment variables
load_dotenv()

False

In [ ]:
# Set up API keys
TOOLHOUSE_API_KEY = "toolhouse_api_key"
OPENAI_API_KEY = "openai_api_key"

# Initialize clients
client = OpenAI(api_key=OPENAI_API_KEY)
th = Toolhouse(api_key=TOOLHOUSE_API_KEY, provider=Provider.OPENAI)

# Model configuration
MODEL = 'gpt-4o'  

In [5]:
def research_topic(topic, depth="medium", sources_count=3):
    """
    Research a topic using Toolhouse tools

    Args:
        topic: The research topic
        depth: "quick", "medium", or "deep"
        sources_count: Number of sources to gather
    """

    # Adjust system prompt based on depth
    depth_instructions = {
        "quick": "Provide a concise summary with key points",
        "medium": "Provide a comprehensive overview with analysis",
        "deep": "Conduct thorough research with detailed analysis and multiple perspectives"
    }

    system_prompt = f"""
    You are a professional research assistant. Your task is to research the topic: "{topic}"

    Research Requirements:
    - Search for {sources_count} different sources
    - {depth_instructions[depth]}
    - Verify information from multiple sources
    - Include relevant data, statistics, and expert opinions
    - Cite sources appropriately

    Use available tools to:
    1. Search for current information
    2. Access relevant websites and documents
    3. Gather diverse perspectives

    Format your response as a structured research report.
    """

    messages = [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": f"Please research: {topic}"}
    ]

    # Get response with tools
    response = client.chat.completions.create(
        model=MODEL,
        messages=messages,
        tools=th.get_tools(),
        max_tokens=2000
    )

    # Process tools and continue
    messages = th.run_tools(response, messages)

    # Get final research report
    final_response = client.chat.completions.create(
        model=MODEL,
        messages=messages,
        max_tokens=2000
    )

    return final_response.choices[0].message.content

In [6]:
research_report = research_topic(
    "Impact of AI on healthcare in 2024",
    depth="medium",
    sources_count=5
)
print(research_report)

The impact of AI on healthcare in 2024 is substantial, with numerous applications and innovations already transforming the industry. Here's a summary of key points gathered from the searches:

1. **Diagnostic Accuracy**: AI enhances diagnostic precision, particularly in analyzing medical images for diseases such as cancer and heart disease. In 2024, AI's capabilities are further advanced, recognizing patterns invisible to the human eye, thus aiding in early and accurate disease diagnosis.

2. **Personalized Medicine**: AI facilitates the creation of individualized treatment plans by processing extensive data, including genetic information and lifestyle factors. This personalization improves treatment efficacy while minimizing side effects.

3. **Administrative Efficiency**: AI automates routine administrative tasks, thereby reducing healthcare professionals' workload and allowing them to focus more on patient care. Applications include scheduling, billing, and managing patient informat

In [7]:
class ResearchPipeline:
    def __init__(self, client, toolhouse):
        self.client = client
        self.th = toolhouse
        self.research_data = {}

    def initial_search(self, topic):
        """Stage 1: Initial broad search"""
        messages = [
            {"role": "system", "content": f"""
            Conduct an initial search on "{topic}".
            Find 3-5 diverse sources and provide a brief overview of each.
            Return the information in JSON format with source URLs and summaries.
            """},
            {"role": "user", "content": f"Search for information about: {topic}"}
        ]

        response = self.client.chat.completions.create(
            model=MODEL,
            messages=messages,
            tools=self.th.get_tools()
        )

        messages = self.th.run_tools(response, messages)

        final_response = self.client.chat.completions.create(
            model=MODEL,
            messages=messages
        )

        return final_response.choices[0].message.content

    def deep_dive_analysis(self, topic, initial_findings):
        """Stage 2: Deep dive into specific aspects"""
        messages = [
            {"role": "system", "content": f"""
            Based on the initial findings, conduct a deeper analysis of "{topic}".

            Initial findings: {initial_findings}

            Focus on:
            1. Recent developments and trends
            2. Expert opinions and analysis
            3. Statistical data and case studies
            4. Future implications

            Scrape relevant pages for detailed information.
            """},
            {"role": "user", "content": "Provide detailed analysis based on the initial findings"}
        ]

        response = self.client.chat.completions.create(
            model=MODEL,
            messages=messages,
            tools=self.th.get_tools()
        )

        messages = self.th.run_tools(response, messages)

        final_response = self.client.chat.completions.create(
            model=MODEL,
            messages=messages,
            max_tokens=1500
        )

        return final_response.choices[0].message.content

    def synthesize_report(self, topic, initial_findings, deep_analysis):
        """Stage 3: Synthesize into final report"""
        messages = [
            {"role": "system", "content": f"""
            Create a comprehensive research report on "{topic}" by synthesizing
            the gathered information.

            Structure the report with:
            1. Executive Summary
            2. Current State Analysis
            3. Key Findings
            4. Trends and Patterns
            5. Future Outlook
            6. Recommendations
            7. Sources and References

            Make it professional and well-organized.
            """},
            {"role": "user", "content": f"""
            Initial findings: {initial_findings}

            Deep analysis: {deep_analysis}

            Please synthesize this into a final research report.
            """}
        ]

        response = self.client.chat.completions.create(
            model=MODEL,
            messages=messages,
            max_tokens=2000
        )

        return response.choices[0].message.content

    def run_full_research(self, topic):
        """Run the complete research pipeline"""
        print(f"🔍 Starting research on: {topic}")

        # Stage 1: Initial search
        print("📋 Stage 1: Initial search...")
        initial_findings = self.initial_search(topic)

        # Stage 2: Deep dive
        print("🔬 Stage 2: Deep dive analysis...")
        deep_analysis = self.deep_dive_analysis(topic, initial_findings)

        # Stage 3: Synthesize
        print("📊 Stage 3: Synthesizing report...")
        final_report = self.synthesize_report(topic, initial_findings, deep_analysis)

        print("✅ Research complete!")
        return final_report

In [8]:
# Example usage
pipeline = ResearchPipeline(client, th)
report = pipeline.run_full_research("Quantum computing breakthroughs in 2024")
print(report)

🔍 Starting research on: Quantum computing breakthroughs in 2024
📋 Stage 1: Initial search...
🔬 Stage 2: Deep dive analysis...
📊 Stage 3: Synthesizing report...
✅ Research complete!
# Quantum Computing Breakthroughs in 2024: A Comprehensive Research Report

## 1. Executive Summary

Quantum computing, once relegated to theoretical research and small-scale demonstrations, has made substantial strides forward in 2024. Notable developments include the advancement of super-pure silicon chips and Google’s revolutionary Willow quantum computing chip. These innovations have significantly enhanced quantum coherence times and error rate reduction, propelling quantum computing closer to practical applications across diverse sectors such as artificial intelligence, secure communication, and drug design. This report analyzes these breakthroughs, examines their implications on the current state of quantum computing, and assesses future directions.

## 2. Current State Analysis

As of 2024, the primar

In [9]:
def compare_topics(topic_a, topic_b, comparison_criteria=None):
    """Compare two topics side by side"""

    if not comparison_criteria:
        comparison_criteria = [
            "Market size and growth",
            "Key players and competitors",
            "Technological advantages",
            "Challenges and limitations",
            "Future prospects"
        ]

    system_prompt = f"""
    Conduct a comparative analysis between "{topic_a}" and "{topic_b}".

    Compare them across these criteria:
    {chr(10).join(f"- {criteria}" for criteria in comparison_criteria)}

    Research both topics thoroughly and provide:
    1. Summary of each topic
    2. Side-by-side comparison table
    3. Key differences and similarities
    4. Advantages and disadvantages of each
    5. Conclusions and recommendations

    Use web search and content analysis tools to gather current information.
    """

    messages = [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": f"Compare {topic_a} vs {topic_b}"}
    ]

    response = client.chat.completions.create(
        model=MODEL,
        messages=messages,
        tools=th.get_tools(),
        max_tokens=2500
    )

    messages = th.run_tools(response, messages)

    final_response = client.chat.completions.create(
        model=MODEL,
        messages=messages,
        max_tokens=2500
    )

    return final_response.choices[0].message.content

In [10]:
# Example comparison
comparison = compare_topics(
    "Electric vehicles",
    "Hydrogen fuel cell vehicles",
    ["Environmental impact", "Infrastructure requirements", "Cost factors", "Performance metrics"]
)
print(comparison)

Based on the gathered information, here is an overview of electric vehicles (EVs) and hydrogen fuel cell vehicles (HFCVs) concerning their environmental impacts, infrastructure, costs, and performance:

**Electric Vehicles (EVs):**
1. **Environmental Impact:**
   - EVs generally have lower lifecycle greenhouse gas (GHG) emissions compared to internal combustion engine vehicles (ICEVs). Most emissions in EVs are concentrated in the battery production and electricity generation stages.
   - The type of electricity used for charging significantly influences the environmental benefits. Charging from renewable sources maximizes GHG reductions.

2. **Infrastructure:**
   - EVs require an extensive charging network. Infrastructure development is advanced in several regions but still lacks in rural and underserved areas.
   - Investments in fast-charging networks and battery technology advancements are crucial to addressing range anxiety and improving adoption rates.

3. **Costs:**
   - The co

In [11]:
class PersistentResearchAssistant:
    def __init__(self, client, toolhouse):
        self.client = client
        self.th = toolhouse
        self.research_memory = []
        self.session_id = None

    def start_research_session(self, session_name):
        """Start a new research session"""
        self.session_id = session_name
        self.research_memory = []
        print(f"Started research session: {session_name}")

    def add_research_context(self, topic, findings):
        """Add research to memory"""
        self.research_memory.append({
            "topic": topic,
            "findings": findings,
            "timestamp": th.get_tools()  # Assuming current_time tool
        })

    def research_with_context(self, topic):
        """Research with awareness of previous research"""

        # Build context from previous research
        context = ""
        if self.research_memory:
            context = "Previous research in this session:\n"
            for item in self.research_memory[-3:]:  # Last 3 items
                context += f"- {item['topic']}: {item['findings'][:200]}...\n"

        system_prompt = f"""
        You are continuing a research session on related topics.

        {context}

        Current research topic: "{topic}"

        Consider connections to previous research and build upon existing findings.
        Use tools to gather new information while referencing relevant previous research.
        """

        messages = [
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": f"Research: {topic}"}
        ]

        response = self.client.chat.completions.create(
            model=MODEL,
            messages=messages,
            tools=self.th.get_tools()
        )

        messages = self.th.run_tools(response, messages)

        final_response = self.client.chat.completions.create(
            model=MODEL,
            messages=messages,
            max_tokens=1500
        )

        findings = final_response.choices[0].message.content
        self.add_research_context(topic, findings)

        return findings

    def generate_session_summary(self):
        """Generate summary of entire research session"""
        if not self.research_memory:
            return "No research conducted in this session."

        topics = [item['topic'] for item in self.research_memory]

        system_prompt = f"""
        Create a comprehensive summary of the research session: {self.session_id}

        Topics researched: {', '.join(topics)}

        Identify:
        1. Common themes across all research
        2. Key insights and patterns
        3. Contradictions or gaps in findings
        4. Recommended follow-up research
        5. Executive summary

        Create a cohesive narrative that connects all the research.
        """

        # Combine all findings
        all_findings = "\n\n".join([
            f"Topic: {item['topic']}\nFindings: {item['findings']}"
            for item in self.research_memory
        ])

        messages = [
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": f"Research findings to summarize:\n\n{all_findings}"}
        ]

        response = self.client.chat.completions.create(
            model=MODEL,
            messages=messages,
            max_tokens=2000
        )

        return response.choices[0].message.content

In [12]:
# Example session
assistant = PersistentResearchAssistant(client, th)
assistant.start_research_session("AI Technology Review")

# Research multiple related topics
topics = [
    "Large Language Models in 2024",
    "AI Ethics and Regulation",
    "AI in Enterprise Applications"
]

for topic in topics:
    print(f"\n🔍 Researching: {topic}")
    findings = assistant.research_with_context(topic)
    print(findings[:300] + "...\n")

# Generate session summary
print("📋 Session Summary:")
summary = assistant.generate_session_summary()
print(summary)

Started research session: AI Technology Review

🔍 Researching: Large Language Models in 2024
The field of Large Language Models (LLMs) has seen significant advancements and innovations in 2024, with a strong focus on enhancing efficiency, multimodal capabilities, specialized models, and collaborative open-source initiatives. Key developments include:

1. **Enhanced Reasoning Abilities**: Mo...


🔍 Researching: AI Ethics and Regulation
In 2024, significant developments are emerging in the realm of AI ethics and regulation. Here are key insights:

1. **EU AI Regulations**: The European Parliament's "Artificial Intelligence Act" provides harmonized rules on AI across the EU, although initially proposed ethical principles were omitte...


🔍 Researching: AI in Enterprise Applications
Artificial Intelligence (AI) is poised to significantly transform enterprise applications in 2024 and beyond. Here are some major trends and predictions:

1. **AI-Driven Decision Making**: AI will be instrument